

Yu Fung David Wang

simulate a simplified airport security system at a busy airport. Passengers arrive according to a Poisson distribution with λ1 = 5 per minute (i.e., mean interarrival rate = 0.2 minutes) to the ID/boarding-pass check queue, where there are several servers who each have exponential service time with mean rate = 0.75 minutes.  After that, the passengers are assigned to the shortest of the several personal-check queues, where they go through the personal scanner (time is uniformly distributed between 0.5 minutes and 1 minute). 

build a simulation of the system, and then vary the number of ID/boarding-pass checkers and personal-check queues to determine how many are needed to keep average wait times below 15 minutes.  Use rate = 50

In [2]:
import simpy
import random

In [7]:
wait_times = [] # to calculate the average time it takes later

''' Create the environment '''
class Airport(object):
    def __init__(self, environ, num_servers, num_scanners):
        self.environ = environ
        self.servers = simpy.Resource(environ, num_servers)
        self.scanners = simpy.Resource(environ, num_scanners)

    ''' setting up the different factors of the environment'''

    def service(self, passenger):
        # mean service time rate of 0.75 minutes to go through service
        yield self.environ.timeout(0.75) 

    def scan(self, passenger):
         # time of scanner is uniformly distributed (random) between 0.5 and 1 minute to go through scanner
        yield self.environ.timeout(random.uniform(0.5, 1))

''' setting up the process of passengers using the airport '''
def passenger_arrival(environ, passenger, airport):
    arrival = environ.now # grab the arrival time

    ''' request of servvers to work the service (function boarding_service())
         the with statement tells the simulation to release the resource once the process is complete 
         (ex: once passenger goes through service, it will leave the boarding section)'''
        
    # passenger generates a request to use the boarding service
    with airport.servers.request() as request: 
        # passenger waits for the servers to be available to be used
        yield request 
        # passenger uses the availaable server to complete the boarding_service process
        yield environ.process(airport.service(passenger)) 

    with airport.scanners.request() as request: # similar idea as above except with the scanner
        yield request
        yield environ.process(airport.scan(passenger)) 

    wait_times.append(environ.now - arrival)

''' create a function to creating instance of airport and generating passengers '''
def airport_process(environ, num_servers, num_scanners): 
    airport = Airport(environ, num_servers, num_scanners)

    passenger = 0
    
    while True:
        ''' 5 passengers arrive in 1 minute so each passenger has a mean interarrival rate of 0.2 minutes 
         hence we wait a bit before generating new passengers '''
        yield environ.timeout(0.2) 
        passenger += 1 # generate new passenger
        environ.process(passenger_arrival(environ, passenger, airport)) # initialize the passenger

''' function to calcualte the average wait time as required by the question ''' 
def average_wait_time(wait_times):
    average = sum(wait_times)/len(wait_times)
    # grab this average to turn into minutes and seconds
    mint, f_mint = divmod(average, 1)
    secs = f_mint * 60
    return round(mint), round(secs)

''' grab the inputs from the user of the simulation '''
def user_input():
    num_servers = input("Please Input the Number of Servers in the ID/Boarding-Pass Check Queue Working: ")
    num_scanners = input("Please Input the Number of Scanners in the Personal Check Queue Working: ")

    # check if the numbers are actually numbers in string (isdigit())
    if num_servers.isdigit() and num_scanners.isdigit():
        workers = (int(num_servers), int(num_scanners))
        return workers

    else:
        print("Error: Please input a number")
        return


''' Main function to tie everything together '''
def main():

    random.seed(123) # set up with a random seed
    num_servers, num_scanners = user_input() # query the user for input

    # run the actual simulation
    # create environment to save into environ to move simulation through steps
    environ = simpy.Environment() 
    # run the simulation main process to create environ and generate passengers
    environ.process(airport_process(environ, num_servers, num_scanners)) 
    # mentioned by the problem where we simulate a busier airport and run to 50 (how many minutes we want it to run for)
    environ.run(until = 50) 

    # results
    minute, second = average_wait_time(wait_times)
    print(f"The average wait time is {minute}'{second}''")

if __name__ == '__main__':
    main()






Please Input the Number of Servers in the ID/Boarding-Pass Check Queue Working:  2
Please Input the Number of Scanners in the Personal Check Queue Working:  2


The average wait time is 12'50''


Based on the random seed of 123 and the other requirements of the question, at least 2 servers and 2 scanners are needed to be working to keep the avereage wait time under 15 minutes as it gives a average wait time of 12 minutes and 50 seconds.

If there is only 1 working server, the average wait time will be 19 minutes and 4 seconds no matter how many scanners there are. Similarly, if there is only 1 working scanner, the average wait time will be 18 minutes and 32 seconds no matter how many servers there are. This is due to how you have to go through both the service and scanning process to get onto the airplane, hence both are dependent on one another. If one process is bottlenecked, the majority of time will be spent on that process. 